In [4]:
import pandas as pd
import numpy as np
import time
import re
import csv
import math
from tqdm.notebook import tqdm
tqdm.pandas()

from exact import solve
from util import lane_order

Solving with each setting of cutting planes.

In [11]:
def eval_exact(data):
    def solve_instance(x, cut):
        start = time.time()
        y = solve(x['instance'], cutting_planes=cut)
        return y['y'], y['obj'], time.time() - start

    for cut in [0,1,2]:
        # apply branch-and-bound with different cutting planes active
        data[[f'opt_y_{cut}', f'opt_obj_{cut}', f'running_time_{cut}']] = \
            data.progress_apply(lambda x: solve_instance(x, cut), axis=1, result_type='expand')

    # we verify the cutting planes by comparing objectives
    data['matching1'] = data.apply(
        lambda x: math.isclose(x['opt_obj_0'], x['opt_obj_1'], rel_tol=1e-04), axis=1)
    data['matching2'] = data.apply(
        lambda x: math.isclose(x['opt_obj_0'], x['opt_obj_2'], rel_tol=1e-04), axis=1)
    assert data['matching1'].all() and data['matching2'].all()

    # rename and remove columns
    data['opt_y'] = data['opt_y_1']
    data['opt_obj'] = data['opt_obj_1']
    data = data.drop([
            'opt_y_0', 'opt_obj_0',
            'opt_y_1', 'opt_obj_1',
            'opt_y_2', 'opt_obj_2',
            'matching1', 'matching2'
       ], axis=1, errors='ignore')

    # store schedules with their lane order
    data['opt_eta'] = data.apply(lambda x: lane_order(x['opt_y']), axis=1)

    return data

Solve optimally for the given instances.

In [12]:
sets = [1,2,3,4,5,6]
files = [f"data/test_{i}.pkl" for i in sets]

for file in files:
    print(f"processing {file}")
    data = pd.read_pickle(file)

    print('branch-and-bound')
    data = eval_exact(data)

    # save everything with pickle
    data.to_pickle(file)

processing data/test_5.pkl
branch-and-bound


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

processing data/test_6.pkl
branch-and-bound


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]